In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ["OPENAI_API_KEY"] = "OPENAI-API-KEY"

import io
import json
import pandas as pd
from typing_extensions import TypedDict, Annotated
from contextlib import redirect_stdout

# LangChain 관련 라이브러리
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor
from langchain_core.runnables import RunnablePassthrough
import gradio as gr

# Hugging Face 관련 임포트
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

/home/ubuntu/anaconda3/envs/ds_ggls/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

####################################
# 1. 도서관 관련 테이블 DDL 정의
####################################

library_schema = """
-- 도서 테이블
CREATE TABLE books (
    book_id VARCHAR(10) PRIMARY KEY,  -- 도서ID (예: B001)
    title VARCHAR(100) NOT NULL,      -- 제목
    author VARCHAR(50) NOT NULL,      -- 저자
    publisher VARCHAR(50) NOT NULL,   -- 출판사
    publication_year INT NOT NULL,    -- 출판년도
    isbn VARCHAR(20) NOT NULL,        -- ISBN
    category VARCHAR(30) NOT NULL,    -- 카테고리
    subcategory VARCHAR(30),          -- 하위 카테고리
    language VARCHAR(20) NOT NULL,    -- 언어
    pages INT NOT NULL,               -- 페이지 수
    price DECIMAL(10,2) NOT NULL,     -- 가격
    description TEXT,                 -- 설명
    shelf_location VARCHAR(20) NOT NULL -- 서가 위치
);

-- 회원 테이블
CREATE TABLE members (
    member_id VARCHAR(10) PRIMARY KEY, -- 회원ID (예: M001)
    name VARCHAR(50) NOT NULL,         -- 이름
    birth_date DATE NOT NULL,          -- 생년월일
    gender CHAR(1) NOT NULL,           -- 성별 (M/F)
    phone VARCHAR(20) NOT NULL,        -- 전화번호
    email VARCHAR(100) NOT NULL,       -- 이메일
    address VARCHAR(200) NOT NULL,     -- 주소
    membership_type VARCHAR(20) NOT NULL, -- 회원유형 (일반/학생/교직원/VIP)
    registration_date DATE NOT NULL,   -- 가입일
    expiration_date DATE NOT NULL,     -- 만료일
    status VARCHAR(20) NOT NULL        -- 상태 (활성/정지/만료)
);

-- 대출 기록 테이블
CREATE TABLE loans (
    loan_id VARCHAR(10) PRIMARY KEY,  -- 대출ID (예: L0001)
    book_id VARCHAR(10) NOT NULL,     -- 도서ID
    member_id VARCHAR(10) NOT NULL,   -- 회원ID
    loan_date DATE NOT NULL,          -- 대출일
    due_date DATE NOT NULL,           -- 반납예정일
    return_date DATE,                 -- 실제 반납일
    status VARCHAR(20) NOT NULL,      -- 상태 (대출중/반납완료/연체)
    extended BOOLEAN NOT NULL,        -- 연장여부
    FOREIGN KEY (book_id) REFERENCES books(book_id),
    FOREIGN KEY (member_id) REFERENCES members(member_id)
);

-- 예약 테이블
CREATE TABLE reservations (
    reservation_id VARCHAR(10) PRIMARY KEY, -- 예약ID (예: R001)
    book_id VARCHAR(10) NOT NULL,          -- 도서ID
    member_id VARCHAR(10) NOT NULL,        -- 회원ID
    reservation_date DATETIME NOT NULL,    -- 예약일시
    expiration_date DATE NOT NULL,         -- 예약만료일
    status VARCHAR(20) NOT NULL,           -- 상태 (대기중/취소/완료)
    FOREIGN KEY (book_id) REFERENCES books(book_id),
    FOREIGN KEY (member_id) REFERENCES members(member_id)
);

-- 도서 상태 테이블
CREATE TABLE book_status (
    book_id VARCHAR(10) NOT NULL,     -- 도서ID
    status VARCHAR(20) NOT NULL,      -- 상태 (대출가능/대출중/예약중/분실/파손/폐기)
    last_updated DATETIME NOT NULL,   -- 최종 업데이트 일시
    notes TEXT,                       -- 비고
    PRIMARY KEY (book_id),
    FOREIGN KEY (book_id) REFERENCES books(book_id)
);

-- 연체 기록 테이블
CREATE TABLE overdue_records (
    record_id VARCHAR(10) PRIMARY KEY, -- 기록ID (예: O001)
    loan_id VARCHAR(10) NOT NULL,      -- 대출ID
    fine_amount DECIMAL(10,2) NOT NULL,-- 연체료
    payment_status VARCHAR(20) NOT NULL,-- 납부상태 (미납/납부완료/면제)
    payment_date DATE,                 -- 납부일
    notes TEXT,                        -- 비고
    FOREIGN KEY (loan_id) REFERENCES loans(loan_id)
);

-- 사서 테이블
CREATE TABLE librarians (
    librarian_id VARCHAR(10) PRIMARY KEY, -- 사서ID (예: LIB001)
    name VARCHAR(50) NOT NULL,          -- 이름
    position VARCHAR(30) NOT NULL,      -- 직책
    department VARCHAR(30) NOT NULL,    -- 부서
    phone VARCHAR(20) NOT NULL,         -- 전화번호
    email VARCHAR(100) NOT NULL,        -- 이메일
    hire_date DATE NOT NULL             -- 고용일
);

-- 도서 구매 내역 테이블
CREATE TABLE book_purchases (
    purchase_id VARCHAR(10) PRIMARY KEY, -- 구매ID (예: P001)
    book_id VARCHAR(10) NOT NULL,        -- 도서ID
    vendor VARCHAR(50) NOT NULL,         -- 공급업체
    purchase_date DATE NOT NULL,         -- 구매일
    purchase_price DECIMAL(10,2) NOT NULL,-- 구매가격
    librarian_id VARCHAR(10) NOT NULL,   -- 담당 사서ID
    notes TEXT,                          -- 비고
    FOREIGN KEY (book_id) REFERENCES books(book_id),
    FOREIGN KEY (librarian_id) REFERENCES librarians(librarian_id)
);

-- 이벤트 테이블
CREATE TABLE events (
    event_id VARCHAR(10) PRIMARY KEY,   -- 이벤트ID (예: E001)
    title VARCHAR(100) NOT NULL,        -- 제목
    description TEXT,                   -- 설명
    event_date DATE NOT NULL,           -- 날짜
    start_time TIME NOT NULL,           -- 시작시간
    end_time TIME NOT NULL,             -- 종료시간
    location VARCHAR(50) NOT NULL,      -- 장소
    max_attendees INT NOT NULL,         -- 최대참석자수
    librarian_id VARCHAR(10) NOT NULL,  -- 담당 사서ID
    status VARCHAR(20) NOT NULL,        -- 상태 (예정/진행중/완료/취소)
    FOREIGN KEY (librarian_id) REFERENCES librarians(librarian_id)
);
"""

####################################
# 2. 도서관 샘플 데이터
####################################

# 도서 샘플 데이터
sample_books = """
INSERT INTO books VALUES('B001', '해리 포터와 마법사의 돌', 'J.K. 롤링', '문학수첩', 1997, '9788983920010', '판타지', '청소년', '한국어', 366, 15000, '해리 포터 시리즈의 첫 번째 책', 'A1-01');
INSERT INTO books VALUES('B002', '노인과 바다', '어니스트 헤밍웨이', '민음사', 1952, '9788937460180', '소설', '고전', '한국어', 132, 12000, '노인 어부의 힘겨운 사투', 'A2-02');
INSERT INTO books VALUES('B003', '1984', '조지 오웰', '문예출판사', 1949, '9788931001341', '소설', '디스토피아', '한국어', 424, 13500, '전체주의 사회를 그린 소설', 'A2-03');
INSERT INTO books VALUES('B004', '죄와 벌', '표도르 도스토예프스키', '열린책들', 1866, '9788932901213', '소설', '고전', '한국어', 768, 18000, '인간의 죄의식과 구원에 관한 소설', 'A2-04');
INSERT INTO books VALUES('B005', '수학의 정석', '홍성대', '성지출판', 2010, '9788988775615', '교과서', '수학', '한국어', 456, 25000, '고등학교 수학 참고서', 'B1-01');
INSERT INTO books VALUES('B006', '데이터베이스 시스템', '김경호', '한빛미디어', 2018, '9788968481475', '전문서적', '컴퓨터과학', '한국어', 540, 32000, '데이터베이스 설계와 구현', 'B3-01');
INSERT INTO books VALUES('B007', '토지', '박경리', '나남', 1986, '9788930085571', '소설', '역사', '한국어', 512, 20000, '한국의 근대화 과정을 그린 대하소설', 'A1-07');
INSERT INTO books VALUES('B008', '광장', '최인훈', '문학과지성사', 1960, '9788932007144', '소설', '현대', '한국어', 202, 10000, '분단 시대를 살아가는 지식인의 고뇌', 'A1-08');
INSERT INTO books VALUES('B009', '구글신은 모든 것을 알고 있다', '이원영', '21세기북스', 2016, '9788950957148', '교양', 'IT', '한국어', 248, 14500, '빅데이터와 인공지능의 시대', 'C1-02');
INSERT INTO books VALUES('B010', '코스모스', '칼 세이건', '사이언스북스', 1980, '9788983711892', '교양', '과학', '한국어', 720, 22000, '우주에 관한 과학 이야기', 'C2-01');
"""

# 회원 샘플 데이터
sample_members = """
INSERT INTO members VALUES('M001', '김민준', '1980-05-15', 'M', '010-1234-5678', 'kim@example.com', '서울시 강남구', '일반', '2020-01-10', '2025-01-09', '활성');
INSERT INTO members VALUES('M002', '이서연', '1992-08-20', 'F', '010-2345-6789', 'lee@example.com', '서울시 서초구', '학생', '2021-03-15', '2023-03-14', '활성');
INSERT INTO members VALUES('M003', '박지훈', '1975-12-10', 'M', '010-3456-7890', 'park@example.com', '서울시 송파구', 'VIP', '2019-05-20', '2024-05-19', '활성');
INSERT INTO members VALUES('M004', '최수아', '1988-03-25', 'F', '010-4567-8901', 'choi@example.com', '서울시 마포구', '일반', '2022-07-30', '2024-07-29', '활성');
INSERT INTO members VALUES('M005', '정도윤', '2000-01-30', 'M', '010-5678-9012', 'jung@example.com', '서울시 영등포구', '학생', '2022-09-01', '2023-08-31', '정지');
"""

# 대출 기록 샘플 데이터
sample_loans = """
INSERT INTO loans VALUES('L0001', 'B001', 'M001', '2023-03-10', '2023-03-24', '2023-03-23', '반납완료', FALSE);
INSERT INTO loans VALUES('L0002', 'B003', 'M002', '2023-03-15', '2023-03-29', NULL, '대출중', FALSE);
INSERT INTO loans VALUES('L0003', 'B005', 'M003', '2023-03-01', '2023-03-15', '2023-03-20', '연체', FALSE);
INSERT INTO loans VALUES('L0004', 'B007', 'M001', '2023-03-23', '2023-04-06', NULL, '대출중', TRUE);
INSERT INTO loans VALUES('L0005', 'B009', 'M004', '2023-03-05', '2023-03-19', '2023-03-19', '반납완료', FALSE);
"""

# 예약 샘플 데이터
sample_reservations = """
INSERT INTO reservations VALUES('R001', 'B002', 'M005', '2023-03-15 14:30:00', '2023-03-22', '대기중');
INSERT INTO reservations VALUES('R002', 'B004', 'M002', '2023-03-10 11:45:00', '2023-03-17', '취소');
INSERT INTO reservations VALUES('R003', 'B006', 'M001', '2023-03-18 16:20:00', '2023-03-25', '완료');
INSERT INTO reservations VALUES('R004', 'B003', 'M003', '2023-03-20 09:10:00', '2023-03-27', '대기중');
INSERT INTO reservations VALUES('R005', 'B010', 'M004', '2023-03-22 13:00:00', '2023-03-29', '대기중');
"""

# 도서 상태 샘플 데이터
sample_book_status = """
INSERT INTO book_status VALUES('B001', '대출가능', '2023-03-23 14:30:00', NULL);
INSERT INTO book_status VALUES('B002', '대출가능', '2023-03-01 10:00:00', NULL);
INSERT INTO book_status VALUES('B003', '대출중', '2023-03-15 11:45:00', NULL);
INSERT INTO book_status VALUES('B004', '대출가능', '2023-03-05 09:30:00', NULL);
INSERT INTO book_status VALUES('B005', '대출가능', '2023-03-20 16:15:00', '연체 반납');
INSERT INTO book_status VALUES('B006', '대출가능', '2023-02-28 13:20:00', NULL);
INSERT INTO book_status VALUES('B007', '대출중', '2023-03-23 15:10:00', NULL);
INSERT INTO book_status VALUES('B008', '대출가능', '2023-03-10 11:00:00', NULL);
INSERT INTO book_status VALUES('B009', '대출가능', '2023-03-19 14:45:00', NULL);
INSERT INTO book_status VALUES('B010', '예약중', '2023-03-22 13:00:00', NULL);
"""

# 연체 기록 샘플 데이터
sample_overdue_records = """
INSERT INTO overdue_records VALUES('O001', 'L0003', 5000.00, '미납', NULL, '5일 연체');
INSERT INTO overdue_records VALUES('O002', 'L0001', 1000.00, '납부완료', '2023-03-25', '1일 연체');
"""

# 사서 샘플 데이터
sample_librarians = """
INSERT INTO librarians VALUES('LIB001', '이지은', '대리', '대출/반납', '02-123-4567', 'jieun@library.org', '2018-03-01');
INSERT INTO librarians VALUES('LIB002', '김태우', '과장', '수서', '02-123-4568', 'taewoo@library.org', '2015-09-15');
INSERT INTO librarians VALUES('LIB003', '박소연', '사원', '참고봉사', '02-123-4569', 'soyeon@library.org', '2021-01-10');
INSERT INTO librarians VALUES('LIB004', '정민호', '부장', '총괄', '02-123-4570', 'minho@library.org', '2010-05-01');
"""

# 도서 구매 내역 샘플 데이터
sample_book_purchases = """
INSERT INTO book_purchases VALUES('P001', 'B001', '교보문고', '2022-12-10', 12000.00, 'LIB002', NULL);
INSERT INTO book_purchases VALUES('P002', 'B002', '예스24', '2022-12-15', 10000.00, 'LIB002', NULL);
INSERT INTO book_purchases VALUES('P003', 'B003', '알라딘', '2023-01-05', 11000.00, 'LIB002', '할인 적용');
INSERT INTO book_purchases VALUES('P004', 'B004', '교보문고', '2023-01-10', 15000.00, 'LIB002', NULL);
INSERT INTO book_purchases VALUES('P005', 'B005', '인터파크', '2023-01-20', 20000.00, 'LIB002', NULL);
"""

# 이벤트 샘플 데이터
sample_events = """
INSERT INTO events VALUES('E001', '어린이 독서 교실', '초등학생 대상 독서 프로그램', '2023-04-05', '14:00', '16:00', '1층 강당', 30, 'LIB003', '예정');
INSERT INTO events VALUES('E002', '작가와의 만남', '신간 도서 출판 기념 저자 특강', '2023-04-10', '19:00', '21:00', '2층 세미나실', 50, 'LIB004', '예정');
INSERT INTO events VALUES('E003', '도서관 이용 교육', '신규 회원 대상 도서관 이용 안내', '2023-03-15', '10:00', '11:00', '1층 로비', 20, 'LIB001', '완료');
INSERT INTO events VALUES('E004', '독서 토론회', '이달의 책 토론', '2023-03-25', '18:00', '20:00', '2층 세미나실', 15, 'LIB003', '예정');
INSERT INTO events VALUES('E005', '영화 상영회', '원작 소설 기반 영화 상영', '2023-04-20', '15:00', '17:30', '지하 1층 시청각실', 40, 'LIB004', '예정');
"""

# 전체 DDL과 샘플 데이터 합치기
library_complete_schema = library_schema + sample_books + sample_members + sample_loans + sample_reservations + sample_book_status + sample_overdue_records + sample_librarians + sample_book_purchases + sample_events


In [3]:


####################################
# 3. SQLite 데이터베이스 설정
####################################

import sqlite3
from langchain_community.utilities import SQLDatabase
import tempfile
import os

# 임시 SQLite 데이터베이스 생성
def create_library_db():
    db_file = os.path.join(tempfile.gettempdir(), "library.db")
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # 기존 테이블 삭제 (이미 존재하는 경우)
    cursor.executescript("""
    DROP TABLE IF EXISTS events;
    DROP TABLE IF EXISTS book_purchases;
    DROP TABLE IF EXISTS librarians;
    DROP TABLE IF EXISTS overdue_records;
    DROP TABLE IF EXISTS book_status;
    DROP TABLE IF EXISTS reservations;
    DROP TABLE IF EXISTS loans;
    DROP TABLE IF EXISTS members;
    DROP TABLE IF EXISTS books;
    """)

    # DDL 및 샘플 데이터 실행
    cursor.executescript(library_schema)
    cursor.executescript(sample_books)
    cursor.executescript(sample_members)
    cursor.executescript(sample_loans)
    cursor.executescript(sample_reservations)
    cursor.executescript(sample_book_status)
    cursor.executescript(sample_overdue_records)
    cursor.executescript(sample_librarians)
    cursor.executescript(sample_book_purchases)
    cursor.executescript(sample_events)

    conn.commit()
    conn.close()

    # LangChain SQL 데이터베이스 인스턴스 반환
    return SQLDatabase.from_uri(f"sqlite:///{db_file}")

# 데이터베이스 생성 실행
db = create_library_db()
print("데이터베이스가 성공적으로 생성되었습니다.")
print(f"사용 가능한 테이블: {db.get_usable_table_names()}")

# 테이블 내용 확인 예시
books_example = db.run("SELECT * FROM books LIMIT 3;")
print(f"도서 테이블 샘플: {books_example}")

####################################
# 4. Text-to-SQL 시스템 구현
####################################

# Qwen 모델 초기화
# Hugging Face Inference API로 모델에 접근
from huggingface_hub import InferenceClient

# 허깅페이스 토큰 설정 (필요한 경우)
# os.environ["HUGGINGFACE_API_KEY"] = "your_huggingface_token"

# Qwen 모델 설정 
# 로컬 모델 사용 방식 (Hugging Face Transformers 사용)
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline # LangChain 파이프라인 생성
tokenizer_sql = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Coder-3B-Instruct", trust_remote_code=True)
model_sql = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-Coder-3B-Instruct", trust_remote_code=True)

# SQL 생성 파이프라인
pipe_sql = pipeline(
    "text-generation",
    model=model_sql,
    tokenizer=tokenizer_sql,
    max_new_tokens=1024,
    temperature=0.01,
    top_p=0.95,
    repetition_penalty=1.1,
    return_full_text=False
)

# SQL 생성용 LLM
llm_sql = HuggingFacePipeline(pipeline=pipe_sql)

# 자연어 응답 생성용 모델 (일반 대화 모델 사용)
# 예: 더 자연스러운 대화를 위한 모델 선택
tokenizer_chat = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct", trust_remote_code=True)
model_chat = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-3B-Instruct", trust_remote_code=True)

# 자연어 생성 파이프라인
pipe_chat = pipeline(
    "text-generation",
    model=model_chat,
    tokenizer=tokenizer_chat,
    max_new_tokens=512,
    temperature=0.7,  # 더 자연스러운 응답을 위해 온도 상향
    top_p=0.9,
    repetition_penalty=1.05,
    return_full_text=False
)

# 자연어 응답용 LLM
from langchain.chat_models import ChatOpenAI
llm_chat = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# SQL 쿼리 생성 함수
def generate_sql(question):
    """사용자 질문으로부터 SQL 쿼리 생성"""
    system_prompt = """
    당신은 도서관 데이터베이스 SQL 전문가입니다.
    사용자의 질문을 받아서 해당 질문에 답하기 위한 SQL 쿼리를 생성해야 합니다.

    다음은 데이터베이스 스키마입니다:
    {schema}

    사용 가능한 테이블:
    {table_names}

    주어진 질문에 대한 정확한 SQL 쿼리를 생성하세요.
    SQL 쿼리만 반환하고 다른 설명은 포함하지 마세요.
    쿼리는 SQLite 문법을 사용해야 합니다.

    중요: 사용자 질문에 이전 대화 내용이 포함된 경우, 맥락에 따라 정확한 SQL 쿼리를 생성하세요.
    예를 들어 "그 책의 다른 대출 기록도 보여줘"와 같은 질문이 있으면,
    이전 대화에서 언급된 책을 참조하여 적절한 SQL 쿼리를 생성해야 합니다.

    만약 질문이 데이터베이스 쿼리로 해결할 수 없는 경우(예: 인사말, 일반적인 대화, 데이터베이스와 무관한 질문),
    'SQL_QUERY_NOT_APPLICABLE'이라고 답변하세요.
    """

    human_prompt = "{question}"

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", human_prompt),
    ])

    filled_prompt = prompt.invoke({
        "schema": library_schema,
        "table_names": ", ".join(db.get_usable_table_names()),
        "question": question
    })

    response = llm_sql.invoke(filled_prompt.to_string())
    # SQL 쿼리만 추출 (```sql과 ``` 제거)
    query = response
    if "```sql" in query:
        query = query.split("```sql")[1].split("```")[0].strip()
    elif "```" in query:
        query = query.split("```")[1].split("```")[0].strip()
    elif "SELECT" in query.upper() or "INSERT" in query.upper() or "UPDATE" in query.upper() or "DELETE" in query.upper():
        # 코드 블록으로 감싸져 있지 않지만 SQL 쿼리의 일부로 보이는 경우
        query = query.strip()
    else:
        query = query.strip()
        
    if "SQL_QUERY_NOT_APPLICABLE" in query:
        return "SQL_QUERY_NOT_APPLICABLE"
        
    return query

# SQL 쿼리 실행 함수
def execute_sql(query):
    """SQL 쿼리 실행"""
    try:
        result = db.run(query)
        return result
    except Exception as e:
        return f"에러 발생: {str(e)}"

# 자연어 답변 생성 함수
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 자연어 응답 생성을 위한 ChatPromptTemplate 정의
answer_system_prompt = """
당신은 도서관 이용자에게 정보를 제공하는 친절한 도서관 안내 도우미입니다.
SQL 쿼리 결과를 바탕으로 사용자의 질문에 자연스럽게 답변해야 합니다.

결과를 해석하여 일반 사용자가 이해하기 쉬운 자연어로 답변하세요.
기술적인 용어나 SQL 관련 내용은 언급하지 마세요.
프로그래밍 코드를 포함하지 마세요.
간결하고 직접적인 답변을 제공하세요.
"""

answer_human_prompt = """
질문: {question}
SQL 쿼리 결과: {result}

위 정보를 바탕으로 질문에 대답해주세요.
"""

answer_chat_prompt = ChatPromptTemplate.from_messages([
    ("system", answer_system_prompt),
    ("human", answer_human_prompt),
])

# 응답 생성 체인 구성
generate_answer_chain = (
    {
        "question": lambda x: x["question"],
        "result": lambda x: x["result"]
    }
    | answer_chat_prompt
    | llm_chat  # 자연어 응답용 LLM
    | StrOutputParser()
)

# generate_answer 함수 수정


def generate_answer(question, query, result):
    """쿼리 결과를 바탕으로 사용자 질문에 자연어로 답변"""
    
    # 체인 실행에 필요한 입력 준비
    chain_input = {
        "question": question,
        "result": result
    }
    
    # 체인을 통해 응답 생성
    answer = generate_answer_chain.invoke(chain_input)
    
    return answer

# 대화 컨텍스트 관리 클래스
class ConversationContext:
    def __init__(self):
        self.history = []  # 대화 이력 저장 [(질문, 쿼리, 결과, 답변), ...]

    def add_interaction(self, question, query, result, answer):
        self.history.append((question, query, result, answer))

    def get_context_str(self):
        if not self.history:
            return ""

        context = "이전 대화 내용:\n"
        for i, (q, _, _, a) in enumerate(self.history[-3:]):  # 최근 3개 대화만 컨텍스트로 사용
            context += f"질문 {i+1}: {q}\n"
            context += f"답변 {i+1}: {a}\n"
        return context

# 대화 컨텍스트 인스턴스 생성
conversation_context = ConversationContext()

# 전체 처리 과정 함수 (컨텍스트 포함)
def process_question(question, use_context=True):
    """질문 처리 전체 프로세스 (대화 컨텍스트 포함)"""
    # 이전 대화 컨텍스트 가져오기
    context_str = conversation_context.get_context_str() if use_context else ""

    # 컨텍스트가 있는 경우 질문에 추가
    if context_str:
        enhanced_question = f"{context_str}\n새로운 질문: {question}"
    else:
        enhanced_question = question

    query = generate_sql(enhanced_question)

    # SQL 쿼리로 해결할 수 없는 질문 처리
    if query.strip() == "SQL_QUERY_NOT_APPLICABLE":
        # 일반적인 답변 생성
        prompt = f"""
        다음은 사용자의 질문입니다:

        질문: {question}

        이 질문은 도서관 데이터베이스 쿼리로 직접 해결할 수 없는 것으로 판단됩니다.
        사용자에게 친절하게 답변해주세요. 만약 도서관 관련 일반 정보를 요청한 것이라면,
        일반적인 정보를 제공하고, 데이터베이스를 통해 더 구체적인 정보를 얻으려면 어떻게 질문해야 할지 안내해주세요.
        """

        response = llm.invoke(prompt)
        answer = response

        # 대화 내용 저장
        conversation_context.add_interaction(question, "적용 불가", "쿼리 없음", answer)

        return "적용 불가", "SQL 쿼리로 해결할 수 없는 질문입니다.", answer

    # 일반적인 SQL 쿼리 실행
    result = execute_sql(query)
    answer = generate_answer(question, query, result)

    # 대화 내용 저장
    conversation_context.add_interaction(question, query, result, answer)

    return query, result, answer

데이터베이스가 성공적으로 생성되었습니다.
사용 가능한 테이블: ['book_purchases', 'book_status', 'books', 'event_participants', 'events', 'librarians', 'loans', 'members', 'overdue', 'overdue_records', 'reservations', 'staff']
도서 테이블 샘플: [('B001', '해리 포터와 마법사의 돌', 'J.K. 롤링', '문학수첩', 1997, '9788983920010', '판타지', '청소년', '한국어', 366, 15000, '해리 포터 시리즈의 첫 번째 책', 'A1-01'), ('B002', '노인과 바다', '어니스트 헤밍웨이', '민음사', 1952, '9788937460180', '소설', '고전', '한국어', 132, 12000, '노인 어부의 힘겨운 사투', 'A2-02'), ('B003', '1984', '조지 오웰', '문예출판사', 1949, '9788931001341', '소설', '디스토피아', '한국어', 424, 13500, '전체주의 사회를 그린 소설', 'A2-03')]


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.20it/s]
Device set to use cuda:0
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.04it/s]
Device set to use cuda:0
/tmp/ipykernel_253031/2227256531.py:108: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_chat = ChatOpenAI(model="gpt-4o-mini", temperature=0)


In [4]:
####################################
# 5. Gradio UI 구현 및 실행
####################################

# Gradio UI 함수
def ui_process_question(question, history):
    query, result, answer = process_question(question, use_context=True)

    # UI에 표시할 내용 처리
    if query == "적용 불가":
        display_query = "SQL 쿼리 해당 없음"
        display_result = "데이터베이스 조회 없음"
    else:
        display_query = query
        display_result = result

    return "", history + [(question, answer)], display_query, display_result

# 대화 초기화 함수
def clear_conversation():
    # 대화 이력 초기화
    conversation_context.history = []
    return [], "", ""

# Gradio 인터페이스 구성 및 실행
def launch_gradio_interface():
    # UI 시작 전에 대화 컨텍스트 초기화
    conversation_context.history = []
    print("대화 컨텍스트가 초기화되었습니다. 깨끗한 상태에서 시작합니다.")

    with gr.Blocks(css="footer {visibility: hidden}") as demo:
        gr.Markdown("# 도서관 데이터베이스 질의 시스템")
        gr.Markdown("도서관 데이터에 관한 도서, 회원, 대출/반납 기록, 예약 등에 대한 질문을 해보세요.")

        with gr.Row():
            with gr.Column(scale=2):
                chatbot = gr.Chatbot(height=400)
                msg = gr.Textbox(label="질문을 입력하세요", placeholder="예: 현재 대출 중인 도서는 몇 권인가요?")
                clear = gr.Button("대화 초기화")
            with gr.Column(scale=1):
                sql_output = gr.Textbox(label="생성된 SQL 쿼리", lines=4)
                result_output = gr.Textbox(label="쿼리 결과", lines=8)

        msg.submit(ui_process_question, inputs=[msg, chatbot], outputs=[msg, chatbot, sql_output, result_output])
        clear.click(clear_conversation, outputs=[chatbot, sql_output, result_output])

        gr.Markdown("""
        ### 예시 질문:
        - 현재 대출 중인 도서는 몇 권인가요?
        - 소설 카테고리에 있는 책 중 가장 페이지가 많은 책은?
        - 김민준 회원이 대출한 책 목록을 보여주세요.
        - 다음 달에 예정된 도서관 이벤트는 무엇인가요?
        - 가장 많은 책을 대출한 회원은 누구인가요?
        - 연체 중인 대출 기록을 모두 보여주세요.
        - 판타지 장르의 책은 몇 권인가요?
        - 가장 최근에 구매한 도서 5권을 알려주세요.
        """)

    # Gradio 인터페이스 실행
    demo.launch(share=True)

if __name__ == "__main__":
    launch_gradio_interface()

/tmp/ipykernel_253031/3854248471.py:37: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=400)


대화 컨텍스트가 초기화되었습니다. 깨끗한 상태에서 시작합니다.
* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://4e3d22c161bc5c15b6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
